# Modélisation et résolution du problème en PLNE

In [26]:
# first import the Model class from docplex.mp
from docplex.mp.model import Model
import numpy as np
import time
import sys

In [67]:
# "jobs" is a matrix where each value is the working time needed for each job on each machine
# jobs[i][j] is the time needed for job j on machine i

def create_job_scheduling_PLNE(jobs):
    nb_jobs = len(jobs[0])
    M = len(jobs)
    
    # create one model instance, with a name
    mdl = Model(name='job_scheduling_v1')
    
    # Creation of the variables
    # Variable name format: x_<numMachine>_<numJob>_<jobPosition>
    # "x_3_2_4 = 1" ===> "the job #2 will be the 4th job to run on the machine #3"
    # (indexes start at 1)
    x = []
    for m in range(M):
        x.append([])
        for j in range(nb_jobs):
            x[m].append([])
            for k in range(nb_jobs):
                x[m][j].append(mdl.binary_var(name="x_%d_%d_%d" % (m+1, j+1, k+1)))
       
    x = np.asarray(x)
    
    # Each job must be processed one time
    for i in range(nb_jobs):
        mdl.add_constraint(np.sum(np.sum(x[:,i,:])) == 1)
        
    # Each machine can only run 1 job at a time
    for m in range(M):
        for i in range(nb_jobs):
            mdl.add_constraint(np.sum(np.sum(x[m,:,i])) <= 1)
    
    # The machines are running in parallel, so the objective is
    # to minimize the working time of the machine that will work longer than the others
    z = mdl.integer_var(name="objective")
    for m in range(M):
        mdl.add_constraint(z >= np.sum(np.dot(np.asarray(jobs[m]), x[m,:,:])))
    
    mdl.minimize(z)
    return mdl, x

# remplacer tous les print par des écritures dans un fichier
def format_solution_PLNE(outputfile, num, jobs, var, sol, tps_exec): 
    M = len(jobs)
    J = len(jobs[0])
    
    file = open(outputfile, "a")
    print("%d,%d,%d" % (num, M,J), file=file)
    print("---", file=file)
    
    for m in range(M):
        start_time = 0
        for p in range(J):
            for j in range(J):
                if sol.get_value(var[m,j,p]) == 1:
                    print("%d,%d,%d,%d" % (m, j, start_time, start_time + jobs[m][j]), file=file)
                    start_time += jobs[m][j]
                    break
    print("---", file=file)
    print("%lf,%d\n" % (tps_exec, sol.get_objective_value()), file=file)

    
def solve_job_scheduling_PLNE(num, jobs, outputfile=False):
    
    t = time.time()
    
    mdl, x = create_job_scheduling_PLNE(jobs)
    sol = mdl.solve()
    
    tps_exec = time.time() - t
    if sol:
        #print(sol)
        if outputfile: 
            format_solution_PLNE(outputfile, num, jobs, x, sol, tps_exec)    

In [28]:
import csv
def test_job_scheduling_PLNE(inputfile, outputfile):
    f = open(outputfile, 'w')
    f.close()
    with open(inputfile, newline='') as csvfile:
        csvreader = csv.reader(csvfile, delimiter=',', quotechar='|')
        jobs = []
        num = 0
        
        for row in csvreader:
            if row:
                if len(row) != 1:
                    values = []
                    for value in row:
                        values.append(int(value))
                    jobs.append(values)
            else:
                #print(jobs)
                #print()
                solve_job_scheduling_PLNE(num, jobs, outputfile)
                num += 1
                jobs = []
                

# Modélisation et résolution du problème en PPC

In [29]:
from config import setup
setup()

In [30]:
import csv

def create_matrix_list_from_csv(inputfile):
    list_of_matrix=[]
    with open(inputfile, 'r') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=',')
        matrix = []
        
        for line in spamreader:
            if line==[] or len(line) == 1:
                list_of_matrix.append(matrix)
                matrix=[]
            else:
                row = []
                for cell in line:
                    row.append(int (cell))
                matrix.append(row)  
                
        if (matrix!=[]):
            list_of_matrix.append(matrix)
        
        return list_of_matrix

In [31]:
from docplex.cp.model import CpoModel
import numpy as np

def create_model(D):

    M=len(D)
    T=len(D[0])
    
    # Create the model
    mdl = CpoModel(name='projetMineure')

    # Create one interval variable per job operation
    x = [[mdl.interval_var(size=D[m][t], optional=True, name="Machine_{}-Job_{}".format(m, t)) for t in range(T)] for m in range(M)]


    # Force no overlap for operations executed on a same machine
    for m in range(M):
        mdl.add(mdl.no_overlap(x[m]))
    
    for t in range(T):
        mdl.add((sum([mdl.presence_of(x[m][t]) for m in range(M)])==1))
    

    # Minimize termination date
    flatten_x = np.array(x).flatten()
    
    mdl.add(mdl.minimize(mdl.max([mdl.end_of(flatten_x[t]) for t in range(len(flatten_x))])))
        
    return mdl, x

In [32]:
import sys


def test_job_scheduling_PPC(inputfile, outputfile):
        
    file = open(outputfile, 'w')

    list_of_matrix = create_matrix_list_from_csv(inputfile)
    id_matrix=0
    for matrix in list_of_matrix:
        if matrix!=[]:
            mdl, x = create_model(matrix)
            sol=mdl.solve()
            if (sol):   
                #id_Matrice
                print(id_matrix, end='', file=file)
                print(",", end='', file=file)

                #Nombre de Machines
                print(len(matrix), end='', file=file)
                print(",", end='', file=file)

                #Nombre de Tâches
                print(len(matrix[0]), file=file)

                #Separation id_Matrice et Matrice 
                print("---", file=file)

                var=sol.get_all_var_solutions()
                for v in var:
                    if not(v.is_absent()):
                        name=v.get_name().split("-")

                        #Numero de la machine
                        print(name[0].split("_")[1]+",", end='', file=file)

                        #Numero de la tache
                        print(name[1].split("_")[1]+",", end='', file=file)

                        #Temps du debut de la tache
                        print(v.get_start(), end='', file=file)     
                        print(",", end='', file=file)

                        #Temps de la fin de la tache
                        print(v.get_end(), file=file)

                #Separation Matrice et Résultats
                print("---", file=file)

                #Temps d'execution
                print(sol.get_solve_time(), end='', file=file)
                print(",", end='', file=file)

                #Resultat Optimal
                print(sol.get_objective_values()[0], file=file)

                print(file=file)


            else:
                print("error: No solution found", file=file)

            id_matrix+=1

# Modélisation et résolution du problème en PPC

## Heuristique Gloutonne

In [33]:
import time

def heuristique_gloutonne(jobs):
    t = time.perf_counter()
    
    M = len(jobs)
    J = len(jobs[0])
    
    jobs_done = []
    solution = []
    tps_exec = []
    for m in range(M):
        solution.append([])
        tps_exec.append(0)
        
    for j in range(J):
        
        min_machine = tps_exec.index(min(tps_exec))
        min_tps_exec =  tps_exec[min_machine]

        min_job = 0
        min_job_time = 100000
        for i in range(J):
            if jobs[min_machine][i] < min_job_time and not i in jobs_done:
                min_job_time = jobs[min_machine][i]
                min_job = i
                
        solution[min_machine].append(min_job)
        jobs_done.append(min_job)
        tps_exec[min_machine] += min_job_time

    program_time = time.perf_counter() - t
    
    #print(solution)
    #print(tps_exec)
    return solution, max(tps_exec), program_time

### Formatage de la solution pour l'algo génétique

In [34]:
def convert_solution_for_Gen(matrix_sol, nb_jobs):
    
    converted_sol=[-1 for i in range(nb_jobs)]
    
    for i in range(len(matrix_sol)):
        for job in matrix_sol[i]:
            converted_sol[job]=i
            
    return converted_sol
            

### Génération de la population de départ pour l'algorithme génétique 
> 50% aléatoire , 50% glouton

In [35]:
import random

def generate_population(Matrix, population_size, seed=22):
    
    random.seed(seed)
    
    nb_machines=len(Matrix)
    nb_task=len(Matrix[0])
    
    sol_glout=convert_solution_for_Gen(heuristique_gloutonne(Matrix)[0], nb_task)
    
    pop_heuristique=int(population_size*0.5)
    
    population=[sol_glout for i in range(pop_heuristique)]
    
    pop_random=population_size-pop_heuristique
        
    for i in range(pop_random):
        individual=[]
        for task in range(nb_task):
            rdm_machine=random.randint(0,nb_machines-1)
            individual.append(rdm_machine)
            
        population.append(individual)
    
    return population

### Calcul du score pour un individu de la population
> C'est la durée totale que mettent tous les jobs à s'éxécuter sur les machines indiquées

In [36]:
def score_individual(Matrix, individual):
    nb_machines=len(Matrix)
    score_machine= [0 for i in range(nb_machines)]
    
    for i in range(len(individual)):
        score_machine[individual[i]]+=Matrix[individual[i]][i]
        
    return max(score_machine)

### Récupération de 50% de la population actuelle
>45% sont ceux ayant le meilleur score

>5% sont séléctionnés aléatoirement

In [37]:
def selection_fitness(Matrix, population):
    sorted_pop=sorted(population, key=lambda individual: score_individual(Matrix, individual))
    
    nb_fittest=int(len(population)*0.45)
    nb_random=int(len(population)*0.5)-nb_fittest
    
    parents=sorted_pop[0:nb_fittest]
    
    for i in range(nb_random):
        rdm=random.randint(nb_fittest+1, len(population)-1)
        parents.append(sorted_pop[rdm])
    
    return list(parents)

### Crossover
> Les parents font un accouplement 2 à 2 (Chaleeeuuur)

> L'enfant généré récupère une partie de chacun de ses parents (la taille de chaque partie étant définie aléatoirement)

In [38]:
def crossover(parents):
    
    children=[]
    for i in range(len(parents)):
        p1=parents[i]
        if (i!=len(parents)-1):
            p2=parents[i+1]
        else :
            p2=parents[0]
        
        rdm=random.randint(1,len(p1)-2)
        child = p1[0:rdm] + p2[rdm:]
        
        children.append(child)

    return list(children)

#crossover(selection_fitness(M, population))
        

### Mutations
> 5% de la nouvelle population (parents+enfants) est mutée

> Les mutés subissent un changement sur l'une des valeurs du vecteur (un des jobs ne s'exécute plus sur la même machine)

In [39]:
def mutator(Matrix, population):
    nb_mutated=int(len(population)*0.05)
    for i in range(nb_mutated):
        rdm_individual=random.randint(0, len(population)-1)
        rdm_job=random.randint(0,len(Matrix[0])-1)
        rdm_machine=random.randint(0, len(Matrix)-1)
        
        population[rdm_individual][rdm_job]=rdm_machine
        
    return list(population)

### The fittest
> Pour le résultat à la fin de l'algorithme général

In [40]:
def the_fittest(Matrix, population):
    sorted_pop=sorted(population, key=lambda individual: score_individual(Matrix, individual))
    
    return sorted_pop[0], score_individual(Matrix, sorted_pop[0])

## Algorithme génétique

In [41]:
def genetic_algo(Matrix, population_size):
    start = time.time()
    population = generate_population(Matrix, population_size)
    for i in range(200):
        parents = selection_fitness(Matrix, population)
        children = crossover(parents)
        population = mutator(Matrix, parents+children)
        
    solution = the_fittest(Matrix, population)
    
    end=time.time()
    duree=end-start
    
    return solution, duree   

## Formatage de la solution

In [42]:
def format_sol_gen(solution, Matrix, file):
    
    vecteur_sol=solution[0][0]
       
    #Nombre de Machines
    print(len(Matrix), end='',file=file)
    print(",", end='',file=file)
            
    #Nombre de Tâches
    print(len(Matrix[0]),file=file)
            
    #Separation id_Matrice et Matrice 
    print("---",file=file)

    time_vector=[0 for i in range(len(Matrix))]
    
    for v in range(len(vecteur_sol)):
        
        #Numero de la machine
        print(vecteur_sol[v], end='',file=file)
        print(",", end='',file=file)

        #Numero de la tache
        print(v, end='',file=file)
        print(",", end='',file=file)

        start = time_vector[vecteur_sol[v]]
        end = start + Matrix[vecteur_sol[v]][v]
        time_vector[vecteur_sol[v]]=end
        
        #Temps du debut de la tache
        print(start, end='',file=file)     
        print(",", end='',file=file)

        #Temps de la fin de la tache
        print(end,file=file)

    #Separation Matrice et Résultats
    print("---",file=file)

    #Temps d'execution
    print(solution[1], end='',file=file)
    print(",", end='',file=file)

    #Resultat Optimal
    print(solution[0][1],file=file)

In [43]:
import sys

def test_job_scheduling_AlgoGen(inputfile, outputfile):
        
    file = open(outputfile, 'w')

    list_of_matrix = create_matrix_list_from_csv(inputfile)
    id_matrix=0
    for matrix in list_of_matrix:
        if matrix!=[]:
            sol=genetic_algo(matrix, 200)
            
            #id_Matrice
            print(id_matrix, end='',file=file)
            print(",", end='',file=file)
            
            format_sol_gen(sol, matrix, file)

            print(file=file)

    id_matrix+=1

# Génération des tests 

In [44]:
from config import setup
setup()

In [45]:
import random 
import csv

def create_test_interactive(file): 
    # Il faut changer la graine pour avoir des nouvelles matrices aléatoires.
    random.seed(40)
    a = random.randint(0,10)
    print(a)

    while True:
        m = input("Combien de machines ? ")
        try:
            m = int(m)
            break
        except ValueError: 
            print("Veuillez enter un entier svp.") 


    while True:
        t = input("Combien de tâches ? ")
        try:
            t = int(t)
            break
        except ValueError: 
            print("Veuillez enter un entier svp.")

    while True:
        c = input("Cout maximal ?")
        try:
            c = int(c)
            break
        except ValueError: 
            print("Veuillez enter un entier svp.")

    while True:
        nb = input("Combien de tests ?")
        try:
            nb = int(nb)
            break
        except ValueError: 
            print("Veuillez enter un entier svp.")

    with open(file, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile, delimiter=',',
                                quotechar='', quoting=csv.QUOTE_NONE)
        for k in range(0,nb): 
            l = [[random.randint(1,c) for j in range(t)] for i in range(m)]
            writer.writerow([k])
            writer.writerows(l)
            writer.writerow([])
            
            
def create_test(file, nb_machines, nb_taches, cout_maximal, nb_tests): 
    # Il faut changer la graine pour avoir des nouvelles matrices aléatoires.
    random.seed(40)

    with open(file, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile, delimiter=',',
                                quotechar='', quoting=csv.QUOTE_NONE)
        for k in range(0,nb_tests): 
            l = [[random.randint(1,cout_maximal) for j in range(nb_taches)] for i in range(nb_machines)]
            writer.writerow([k])
            writer.writerows(l)
            writer.writerow([])           


# Vérification des solutions

In [62]:
# Sur la meme machine, pas de superposition, ok 
# Toutes les tâches sont faites une fois
# Vérifier le résultat optimal


def check_solution(num_matrix, matrix, score, nb_task):
    #Check if there is an overlaping on the given matrix
    matrix[0][0]
    currMachine = -1
    jobs = []
    currMachine = matrix[0][0]
    admissible = True
    for i in matrix:
        debut = i[2]
        fin = i[3]
        if i[0] == currMachine:
            jobs.append((debut,fin))
        elif i[0] != currMachine:
            admissible = admissible and check_machine(jobs)
            if not admissible:
                print("Overlapping sur matrice ", num_matrix)
            jobs=[(debut,fin)]
            currMachine = i[0]
    #For the last machine
    admissible = admissible and check_machine(jobs)
    if not admissible:
        print("Overlapping sur matrice ", num_matrix)
        return admissible
    
    #Check if all the tasks have been executed exactly once
    by_task = sorted(matrix, key=lambda line: int(line[1]))
    for i,j in zip(by_task, range(nb_task)):
        if int(i[1]) != int(j): 
            print("il manque ",i[1])
            admissible = admissible and False
            
    if not admissible: 
        print("Il manque une tache dans la matrice", num_matrix)
        return admissible
    
    #Check if the optimal result found is the max of the end date.
    last = max([i[3] for i in matrix])
    admissible = admissible and (last == score)
    if not admissible: 
        print("Résultat incorrect dans la mtrice", num_matrix)
        return admissible
    
    return admissible

#check if jobs do not overlap on a single machine
def check_machine(jobs): 
    jobs = sorted(jobs, key=lambda job: job[0])
    for i in range(len(jobs)):
        if i != (len(jobs)-1):
            if jobs[i][0] <= jobs[i+1][0] and jobs[i][1] > jobs[i+1][0]:
                return False            
    return True
        

In [58]:
class ReadingError(Exception):
    pass

def build_summaries(csvfile):
    output = []
    summary = {}
    with open(csvfile, newline='') as csvfile:
            reader = csv.reader(csvfile, delimiter=',', quotechar='', quoting=csv.QUOTE_NONE)
            matrix = []
            debut = False
            fin = False
            res = None
            i=0
            for row in reader:
                i=i+1
                if len(row) == 0 : 
                    summary['matrix'] = matrix
                    output.append(summary)
                    summary = {}
                    matrix = []
                    res = None 
                    debut = False
                    fin = False 
                elif not debut and not fin and len(row) == 3 and row[0] != "---" :
                    summary['num_matrix'] = int(row[0])
                    summary['nb_task'] = int(row[1])
                    summary['nb_machine'] = int(row[2])
                elif not debut and not fin and len(row) == 1 and row[0] == "---": 
                    debut = True
                elif debut:
                    if row[0] == "---":
                        fin = True
                    elif len(row) == 4:
                        matrix.append(row)
                    elif fin:
                        if len(row) == 2:
                            summary['execution_time'] = row[0]
                            summary['score'] = row[1]
                        else :
                            raise ReadingError("Erreur à la lecture du temps d'exécution et du résultat du solveur, ligne : ", row)
                    else:
                        raise ReadingError("Le fichier est mal formé, ligne : ", row)
                else : 
                    raise ReadingError("Le fichier est très mal formé, ligne : ", row, i)
    return output 

# Comparaison de PLNE, PPC, AlgoGen selon différentes instances de tests:

### Jeu de test : 1000 tests avec 10 taches, 10 machines et un cout maximal de 10.

In [64]:
import numpy as np

def benchmark(nb_machines, nb_taches, cout_max, nb_tests):
    
    print("Benchmark de ", nb_tests, " tests pour ", nb_machines, "machines, ", nb_taches, "taches et un cout maximal de ", cout_max)
    
    summary_test = {}
    summary_test['nb_machines'] = nb_machines
    summary_test['nb_taches'] = nb_taches
    summary_test['cout_max'] = cout_max
    summary_test['nb_tests'] = nb_tests
    
    print("    - Creation du fichier de test...")
    testfile = "tests_" + str(nb_machines) + "_" + str(nb_taches) + "_" + str(cout_max) + "_" + str(nb_tests) + ".csv"
    summary_test['testfile'] = testfile
    create_test(testfile, nb_machines=nb_machines, nb_taches=nb_taches, cout_maximal=cout_max, nb_tests=nb_tests)
    print("    - Done.")
    
    print("    - Génération des solutions en PPC...")
    solution_PPC = "solutions_" + str(nb_machines) + "_" + str(nb_taches) + "_" + str(cout_max) + "_" + str(nb_tests) + "_PPC.csv"
    summary_test['solution_PPC'] = solution_PPC
    test_job_scheduling_PPC(testfile, solution_PPC)
    print("    - Done.")
    
    print("    - Génération des solutions en PLNE...")
    solution_PLNE = "solutions_" + str(nb_machines) + "_" + str(nb_taches) + "_" + str(cout_max) + "_" + str(nb_tests) + "_PLNE.csv"
    summary_test['solution_PLNE'] = solution_PLNE
    test_job_scheduling_PLNE(testfile, solution_PLNE)
    print("    - Done.")
    
    print("    - Génération des solutions en AlgoGen...")
    solution_AlgoGen = "solutions_" + str(nb_machines) + "_" + str(nb_taches) + "_" + str(cout_max) + "_" + str(nb_tests) + "_AlgoGen.csv"
    summary_test['solution_AlgoGen'] = solution_AlgoGen
    test_job_scheduling_AlgoGen(testfile, solution_AlgoGen)
    print("    - Done.")
    
    print("    - Vérification des solution de PPC...")
    list_summaries_PPC = build_summaries(solution_PPC)
    nb_admissible_PPC = 0
    for summary in list_summaries_PPC:
        admissible = check_solution(summary['num_matrix'], summary['matrix'], summary['score'], nb_taches)
        summary['admissible'] = admissible
        if admissible: 
            nb_admissible_PPC += 1
        
    tx_admissible_PPC = nb_admissible_PPC/len(list_summaries_PPC)
    summary_test['tx_admissible_PPC'] = tx_admissible_PPC
    print("    - Done.")
    
    
    print("    - Vérification des solution de PLNE...")
    list_summaries_PLNE = build_summaries(solution_PLNE)
    nb_admissible_PLNE = 0
    for summary in list_summaries_PLNE:
        admissible = check_solution(summary['num_matrix'], summary['matrix'], summary['score'], nb_taches)
        summary['admissible'] = admissible
        if admissible:
            nb_admissible_PLNE += 1
            
    tx_admissible_PLNE = float(nb_admissible_PLNE)/float(len(list_summaries_PLNE))
    summary_test['tx_admissible_PLNE'] = tx_admissible_PLNE
    print("    - Done.")
    
    print("    - Vérification des solution de AlgoGen...")
    list_summaries_AlgoGen = build_summaries(solution_AlgoGen)
    nb_admissible_AlgoGen = 0
    for summary in list_summaries_AlgoGen:
        admissible = check_solution(summary['num_matrix'], summary['matrix'], summary['score'], nb_taches)
        summary['admissible'] = admissible
        if admissible:
            nb_admissible_AlgoGen += 1
            
    tx_admissible_AlgoGen = float(nb_admissible_AlgoGen)/float(len(list_summaries_AlgoGen))
    summary_test['tx_admissible_AlgoGen'] = tx_admissible_AlgoGen
    print("    - Done.")
    
    print("    - Temps de calcul :")
    temps_PPC = [float(i['execution_time']) for i in list_summaries_PPC]
    execution_time_PPC = np.mean(temps_PPC)
    summary_test['execution_time_PPC'] = execution_time_PPC
    print("    - Temps moyen d'exécution pour PPC : ", execution_time_PPC)
    temps_PLNE = [float(i['execution_time']) for i in list_summaries_PLNE]
    execution_time_PLNE = np.mean(temps_PLNE)
    summary_test['execution_time_PLNE'] = execution_time_PLNE
    print("    - Temps moyen d'exécution pour PLNE : ", execution_time_PLNE)
    
    temps_AlgoGen = [float(i['execution_time']) for i in list_summaries_AlgoGen]
    execution_time_AlgoGen = np.mean(temps_AlgoGen)
    summary_test['execution_time_AlgoGen'] = execution_time_AlgoGen
    print("    - Temps moyen d'exécution pour AlgoGen : ", execution_time_AlgoGen)
    
    return summary_test

In [65]:
# Dans un summary_test, on a :
# nb_machines, nb_taches, cout_max, nb_tests, testfile, solution_PPC, solution_PLNE, 
# tx_admissible_PPC, tx_admissible_PLNE, execution_time_PPC, execution_time_PLNE


In [68]:
benchmark(10,10,10,1)

Benchmark de  1  tests pour  100 machines,  100 taches et un cout maximal de  10
    - Creation du fichier de test...
    - Done.
    - Génération des solutions en PPC...
    - Done.
    - Génération des solutions en PLNE...
    - Done.
    - Génération des solutions en AlgoGen...
    - Done.
    - Vérification des solution de PPC...
    - Done.
    - Vérification des solution de PLNE...
    - Done.
    - Vérification des solution de AlgoGen...
    - Done.
    - Temps de calcul :
    - Temps moyen d'exécution pour PPC :  2.26321554184
    - Temps moyen d'exécution pour PLNE :  216.554391
    - Temps moyen d'exécution pour AlgoGen :  1.16569399834


{'cout_max': 10,
 'execution_time_AlgoGen': 1.165693998336792,
 'execution_time_PLNE': 216.55439100000001,
 'execution_time_PPC': 2.2632155418395996,
 'nb_machines': 100,
 'nb_taches': 100,
 'nb_tests': 1,
 'solution_AlgoGen': 'solutions_100_100_10_1_AlgoGen.csv',
 'solution_PLNE': 'solutions_100_100_10_1_PLNE.csv',
 'solution_PPC': 'solutions_100_100_10_1_PPC.csv',
 'testfile': 'tests_100_100_10_1.csv',
 'tx_admissible_AlgoGen': 1.0,
 'tx_admissible_PLNE': 1.0,
 'tx_admissible_PPC': 1.0}

#### Evaluation des temps de calcul moyens : 

### 10 Instances les plus compliquées : 

In [42]:
def find_top_10_hardest_problems(list_summaries):
    test = sorted(list_summaries, key=lambda job: job['execution_time'])
    return test[-10:]
    
list_top_10_hardest_problems_PLNE = find_top_10_hardest_problems(list_summaries_PLNE)
num_PLNE = [i['num_matrix'] for i in list_top_10_hardest_problems_PLNE]
num_PLNE = sorted(num_PLNE)
print(num_PLNE)

list_top_10_hardest_problems_PPC = find_top_10_hardest_problems(list_summaries_PPC)
num_PPC = [i['num_matrix'] for i in list_top_10_hardest_problems_PPC]
num_PPC = sorted(num_PPC)
print(num_PPC)

[120, 158, 161, 233, 280, 299, 573, 628, 681, 753]
[0, 1, 122, 254, 316, 410, 643, 864, 872, 942]


## Recuit simulé : 